# Synopsis-to-Vector

In [1]:
import numpy as np
import pandas as pd
import pickle
import math

## Load Pre-trained CBOW Parameters

In [2]:
pkl_file = 'cbow_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

## Get TF-IDF from synopsis.csv

### Load Cleaned Synopsis Data

In [3]:
synopsis = pd.read_csv('../clean_data/synopsis.csv')
synopsis.head()

,MAL_ID,Synopsis
0,1,solar leaving surface planet earth solar polic...
1,5,day life crew routine interrupted chasing targ...
2,6,head reason waste oppose entire cities fun tit...
3,7,individuals powers mind control robin craft us...
4,8,dark century people suffering rule manipulate ...


### Get Word Set from Synopsis Data

In [4]:
syns = synopsis.Synopsis.str.findall('\w+')

vocab = set()
for syn in syns:
    vocab.update(syn)

vocab = sorted(list(vocab))
vocab_size = len(vocab)

### Get TF(Term Frequency)

In [5]:
def get_tf(word, syn):
    # Term Frequency
    return syn.count(word)

    
tf = []
for syn in syns:
    tf_syn = []
    for word in vocab:
        tf_syn.append(get_tf(word, syn))
    tf.append(tf_syn)

tf = np.asarray(tf)
tf.shape

(6000, 5921)

### Get IDF(Inverse Document Frequency)

In [6]:
def get_idf(term):
    # Inverse Document Frequency
    _df = 0
    for syn in syns:
        _df += int(word in syn)
    return math.log(vocab_size/(_df))


idf = []
for word in vocab:
    idf.append(get_idf(word))

idf = np.asarray(idf)
idf.shape

(5921,)

### Get TF-IDF Table

In [7]:
tf_idf = pd.DataFrame(tf * idf, columns=vocab)

## Generate Sentence Vector of Synopsis

In [8]:
def get_word_vec(word):
    return word_vecs[word_to_id[word]]

def get_tf_idf(word, i):
    return tf_idf.loc[i, word]

def get_sent_vec(i):
    sent_vec = np.zeros(100, dtype=np.float16)

    for word in set(syns[i]):
        # sent_vec += get_word_vec(word) * get_tf_idf(word, i)
        sent_vec += get_word_vec(word)
    
    return sent_vec / len(set(syns[i]))

In [9]:
full_sent_vec = []
for i in range(len(syns)):
    full_sent_vec.append(get_sent_vec(i))
    
sent_vec_df = pd.DataFrame(full_sent_vec)
sent_vec_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.068298,-0.725098,0.113098,0.290283,0.453369,0.393066,-0.391357,-0.298828,-0.325928,0.187866,...,0.264160,-0.158203,-0.241699,-0.162842,0.262939,0.133911,0.023285,0.187500,0.445068,0.385498
1,0.421631,-0.444580,0.062561,0.541016,0.240845,0.596191,-0.235229,-0.553223,-0.411377,-0.082825,...,0.140869,-0.198486,-0.248657,-0.059326,0.421631,0.312012,0.078186,0.145752,0.477539,0.286621
2,-0.009071,-0.538086,0.002262,0.397949,0.303467,0.380127,-0.252930,-0.568359,-0.080444,-0.015587,...,-0.218140,-0.153564,-0.535645,-0.525879,0.425293,0.391357,0.175781,0.288330,0.453613,0.252197
3,0.189819,-0.184326,0.163452,0.218872,0.219849,0.500977,-0.290527,-0.662109,-0.368164,0.322754,...,-0.125244,-0.370850,-0.436523,-0.446289,0.511719,0.118896,-0.155273,0.532715,0.250244,0.290283
4,0.280273,-0.597168,0.108032,0.311523,0.265381,0.325928,0.149902,-0.320557,-0.182861,0.143066,...,0.358154,-0.302734,-0.400146,-0.795410,0.366455,0.244995,0.036255,0.251465,0.431885,0.546875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,0.181396,-0.272949,0.447754,0.489502,0.348145,0.204102,-0.381592,-0.229858,-0.260986,0.266602,...,0.186401,-0.143311,-0.356445,-0.459473,0.102417,0.357178,0.234985,0.367432,0.498047,0.644043
5996,0.448242,-0.371826,0.133667,0.429443,0.087402,-0.019547,-0.156006,-0.514160,-0.456055,0.262695,...,-0.156494,-0.344971,-0.471436,-0.486572,0.370605,0.166382,-0.203003,0.529297,0.516602,0.348389
5997,0.331543,-0.499023,0.189941,0.179565,0.204346,0.544434,-0.240479,-0.206177,-0.479492,0.364990,...,0.243408,-0.009750,-0.561035,-0.166626,0.680664,0.229248,0.314209,0.059845,0.658203,0.224487
5998,0.211304,-0.380615,-0.018204,0.232056,0.271973,0.241333,-0.181152,-0.485840,-0.179199,0.503418,...,0.464111,0.237427,-0.382812,-0.260010,0.260742,0.196899,-0.095276,0.191772,0.634766,0.484619


In [10]:
sent_vec_df.insert(0, 'MAL_ID', synopsis.MAL_ID)
sent_vec_df

,MAL_ID,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,1,0.068298,-0.725098,0.113098,0.290283,0.453369,0.393066,-0.391357,-0.298828,-0.325928,...,0.264160,-0.158203,-0.241699,-0.162842,0.262939,0.133911,0.023285,0.187500,0.445068,0.385498
1,5,0.421631,-0.444580,0.062561,0.541016,0.240845,0.596191,-0.235229,-0.553223,-0.411377,...,0.140869,-0.198486,-0.248657,-0.059326,0.421631,0.312012,0.078186,0.145752,0.477539,0.286621
2,6,-0.009071,-0.538086,0.002262,0.397949,0.303467,0.380127,-0.252930,-0.568359,-0.080444,...,-0.218140,-0.153564,-0.535645,-0.525879,0.425293,0.391357,0.175781,0.288330,0.453613,0.252197
3,7,0.189819,-0.184326,0.163452,0.218872,0.219849,0.500977,-0.290527,-0.662109,-0.368164,...,-0.125244,-0.370850,-0.436523,-0.446289,0.511719,0.118896,-0.155273,0.532715,0.250244,0.290283
4,8,0.280273,-0.597168,0.108032,0.311523,0.265381,0.325928,0.149902,-0.320557,-0.182861,...,0.358154,-0.302734,-0.400146,-0.795410,0.366455,0.244995,0.036255,0.251465,0.431885,0.546875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,48438,0.181396,-0.272949,0.447754,0.489502,0.348145,0.204102,-0.381592,-0.229858,-0.260986,...,0.186401,-0.143311,-0.356445,-0.459473,0.102417,0.357178,0.234985,0.367432,0.498047,0.644043
5996,48466,0.448242,-0.371826,0.133667,0.429443,0.087402,-0.019547,-0.156006,-0.514160,-0.456055,...,-0.156494,-0.344971,-0.471436,-0.486572,0.370605,0.166382,-0.203003,0.529297,0.516602,0.348389
5997,48470,0.331543,-0.499023,0.189941,0.179565,0.204346,0.544434,-0.240479,-0.206177,-0.479492,...,0.243408,-0.009750,-0.561035,-0.166626,0.680664,0.229248,0.314209,0.059845,0.658203,0.224487
5998,48483,0.211304,-0.380615,-0.018204,0.232056,0.271973,0.241333,-0.181152,-0.485840,-0.179199,...,0.464111,0.237427,-0.382812,-0.260010,0.260742,0.196899,-0.095276,0.191772,0.634766,0.484619


In [11]:
sent_vec_df.to_csv('synopsis_vector.csv', index=False)